In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def apply_changes(arr,base,base_no):
    '''
    Input Arguments:
    arr = 2D array (n,3) of the station readings taken between the two base readings (n = no. of stations)
           columns of arr = ['Staions','Time','Gravity Readings']
    base = 2D array (b,2) of the base readings at  different time (b = no. of time readings taken at the base)
           columns of base = ['Time','Gravity Readings']
    base_no = the index of last measured base value (0 indexed)
    The function apply the drift correction (creeping correction of the instrument), by applying the following formula:
    g_updated = g_obtained - (slope*time_elapsed)
    where,
    slope is calculated using following formula:
     slope = (base_value1-base_value0)/(time1-time0)
    and, time_elapsed = time differnce between the current station reading time and initial base reading time.
    '''
    t0 = np.datetime64(base[base_no-1,0])
    b0 = base[base_no-1,1]
    t1 = np.datetime64(base[base_no,0])
    b1 = base[base_no,1]
    slope = (b1-b0)/(t1-t0).astype('int')
    time = arr[:,1].astype('datetime64')
    values = arr[:,2]
    init_time = np.datetime64(base[0,0])
    delta_time = (time-init_time).astype('int')
    print(delta_time)
    values = values - slope*delta_time
    print(values)
    return (values)

In [ ]:
def plot_drift_curve(bases):
    '''
    plots a subplot 1st of the three, depicting the drift curve where,
    x-axis = time
    y-axis = base readings
    '''
    plt.subplot(1,3,1)
    plt.plot(bases[:,0],bases[:,1])
    plt.title('Drift Curve (Linear)')
    plt.xlabel('Time')
    plt.ylabel('Base Readings (mGal)')

In [ ]:
def plot_updated_proile(arr):
    '''
    plots a subplot 2nd of the three, depicting the updated gravitaional where,
    x-axis = stations
    y-axis = gravity value reading (mGal)
    '''
    plt.subplot(1,3,3)
    plt.plot(arr[:,0],arr[:,2])
    plt.title('Updated Gravitatonal Profile')
    plt.xlabel('Stations')
    plt.ylabel('g (mGal)')

In [ ]:
def plot_old_profile(arr):
    '''
    plots a subplot 3rd of the three, depicting the obtained (raw) profile where,
    x-axis = stations
    y-axis = gravity value reading (mGal)
    '''
    plt.subplot(1,3,2)
    plt.plot(arr[:,0],arr[:,2])
    plt.title('Obtained Gravitatonal Profile')
    plt.xlabel('Stations')
    plt.ylabel('g (mGal)')

In [ ]:
#import the data from the excel file and perform the necassary preprocessing
df = pd.read_csv('../input/creeping-dataset-csv/data.csv')
df['Gravity reading'] = df['Gravity reading'].astype('float')

In [ ]:
#create the 'base' array that would contain the time and the readings of the base location
base0 = np.array(df.iloc[0,1:])
base1 = np.array(df.iloc[10,1:])
base2 = np.array(df.iloc[19,1:])
bases = np.concatenate([base0,base1,base2],axis=0)
bases = bases.reshape((bases.shape[0]//2,2))

In [ ]:
#create arrays 'arr0' and 'arr1' that would have station readings with time, between the base 1 & base 2
#and base 2 & base3 reaspectively.
arr0 =  np.array(df.iloc[1:10,:])
arr1 = np.array(df.iloc[11:19,:])
obtained_data = np.concatenate([arr0,arr1],axis=0)

In [ ]:
#update the gravitaional accelaration data using 'apply_changes' function 
#make it a single array by concatenating the two arrays
arr0[:,2] = apply_changes(arr0,bases,1)
arr1[:,2] = apply_changes(arr1,bases,2)
updated_arr = np.concatenate([arr0,arr1],axis=0)

In [ ]:
#plot the drift curves and the updated gravitaional profile.
#show the plots
plt.figure(figsize=(60,30))
plot_drift_curve(bases)
plot_old_profile(obtained_data)
plot_updated_proile(updated_arr)
plt.show()